# Estimating the Brock and Mirman (1972) model by SMM

A simplified set of characterizing equations of the Brock and Mirman model are the following.

$$(c_t)^{-1}-\beta E[r_{t+1}(c_{t+1})^{-1}]=0$$
$$c_t+k_{t+1}-w_t-r_tk_t=0$$
$$w_t-(1-\alpha)e^{z_t}(k_t)^{\alpha}=0$$
$$r_t-\alpha e^{z_t}(k_t)^{\alpha-1}=0$$
$$z_t=\rho z_{t-1}+(1-\rho)\mu+\epsilon_t$$
$$\text{where} \quad E[\epsilon_t]=0$$
$$y_t=e^{z_t}(k_t)^{\alpha}$$
$$k_{t+1}=\alpha \beta e^{z_t}k_t^{\alpha}$$

The variable $c_t$ is the aggregate consumption in period $t$, $k_{t+1}$ is the total household savings and investment in period $t$ for which they receive a return in the next period (this model assumes full depreciation of capital). The wage per unit of labor in period $t$ is $w_t$ and the interest rate or rate of return on investment is $r_t$. Total factor productivity is $z_t$ which follows an AR(1) process. $y_t$ is GDP. The rest of the symbols in the equations are parameters that must be estimated $(\alpha, \beta, \rho, \mu, \sigma)$. The constraints on these parameters are the following.
$$\alpha \in [0.01,0.99], \beta=0.99, \rho \in [-0.99,0.99], \mu \in [5,14], \sigma \in [0.01,1.1]$$
Assume that $z_0=\mu$ and that $k_1=mean(k_t)$ from the data. We use the following six moments from the 100 periods of empirical data $\{ c_t, k_t, w_t, r_t, y_t \}_{t=1}^{100}$: mean($c_t$), mean($k_t$), mean($c_t/y_t$), var(y_t), corr($c_t,c_{t-1}$), and corr($c_t,k_t$).
We report the solution $\hat{\theta}=(\hat{\alpha}, \hat{\rho}, \hat{\mu}, \hat{\sigma})$, the vector of moment differences at the optimum, and the criterion function value.

In [1]:
# Import program libraries
import numpy as np
import scipy.stats as sts
import scipy.optimize as opt

# Import the dataset and assign variables
data = np.loadtxt('NewMacroSeries.txt', delimiter = ',')
c_t = data[:,0]
k_t = data[:,1]
w_t = data[:,2]
r_t = data[:,3]
y_t = data[:,4]

In [2]:
# Define a function that draws N x S epsilon from the normal distribution
def norm_draws(unif_vals, sigma):
    norm_draws = sts.norm.ppf(unif_vals, loc=0, scale=sigma)
    return norm_draws

In [3]:
# Create simulated data
unif_vals = sts.uniform.rvs(0, 1, size=(101, 1000))

beta = 0.99

# Simulate z_s by Equation (5) and z_0 = mu
def eqn5(eps, rho, mu):
    z_s = np.empty_like(eps)
    z_s[0][:] = mu
    for t in range(1, 101):
        z_s[t] = rho* z_s[t-1] + (1-rho)*mu + eps[t-1]
        
    return z_s

# Simulate k_s by Equation (7) and k_1 = mean(k_t)
def eqn7(z_s, eps, alpha, beta):
    k_s = np.empty_like(eps)
    k_s[0][:] = k_t.mean()
    for t in range(1, 101):
        k_s[t] = alpha * beta * np.exp(z_s[t]) * (k_s[t-1] ** alpha)
    
    return k_s

# Simulate w_s by Equation (3)
def eqn3(z_s, k_s, alpha):
    w_s = (1-alpha) * np.exp(z_s[1:101]) * (k_s[0:100] ** alpha)
    return w_s

# Simulate r_s by Eqaution (4)
def eqn4(z_s, k_s, alpha):
    r_s = alpha * np.exp(z_s[1:101]) * (k_s[0:100] ** (alpha -1))
    return r_s

# Simulate c_s by Equation (2)
def eqn2(w_s, r_s, k_s):
    c_s = w_s + (r_s * k_s[0:100]) - k_s[1:101]
    return c_s

# Simulate y_s by Equation (6)
def eqn6(z_s, k_s, alpha):
    y_s = np.exp(z_s[1:101]) * (k_s[0:100] ** alpha)
    return y_s

In [4]:
# Create data moments
def data_moments(c, k, y):
    mom1 = c.mean(axis=0)
    mom2 = k.mean(axis=0)
    a = c / y
    mom3 = a.mean(axis=0)
    mom4 = y.var(axis=0)
    cov1 = np.corrcoef(c[1:100], c[0:99])
    mom5 = cov1[0,1]
    cov2 = np.corrcoef(c, k)
    mom6 = cov2[0,1]
    return mom1, mom2, mom3, mom4, mom5, mom6

# Create model moments
def sim_moments(unif_vals, alpha, rho, mu, sigma):
    eps = norm_draws(unif_vals, sigma)
    z_s = eqn5(eps, rho, mu)
    k_s = eqn7(z_s, eps, alpha, beta)
    w_s = eqn3(z_s, k_s, alpha)
    r_s = eqn4(z_s, k_s, alpha)
    c_s = eqn2(w_s, r_s, k_s)
    y_s = eqn6(z_s, k_s, alpha)
    mom1 = c_s.mean(axis=0)
    mom2 = k_s.mean(axis=0)
    a = c_s / y_s
    mom3 = a.mean(axis=0)
    mom4 = y_s.var(axis=0)
    mom5 = np.empty_like(mom1)
    for j in range (0, 1000):
        cov1 = np.corrcoef(c_s[1:100, j], c_s[0:99, j])
        mom5[j] = cov1[0,1]
    mom6 = np.empty_like(mom1)
    for j in range (0, 1000):
        cov2 = np.corrcoef(c_s[:, j], k_s[0:100, j])
        mom6[j] = cov2[0,1]
    return mom1, mom2, mom3, mom4, mom5, mom6

In [5]:
# Create the error vector
def err_vec(c_t, k_t, y_t, unif_vals, alpha, rho, mu, sigma):
    data_mom1, data_mom2, data_mom3, data_mom4, data_mom5, data_mom6 
    = data_moments(c_t, k_t, y_t)
    moms_data = np.array([data_mom1, data_mom2, data_mom3, data_mom4, 
                          data_mom5, data_mom6])
    sim_mom1, sim_mom2, sim_mom3, sim_mom4, sim_mom5, sim_mom6 
    = sim_moments(unif_vals, alpha, rho, mu, sigma)
    mod_mom1 = sim_mom1.mean()
    mod_mom2 = sim_mom2.mean()
    mod_mom3 = sim_mom3.mean()
    mod_mom4 = sim_mom4.mean()
    mod_mom5 = sim_mom5.mean()
    mod_mom6 = sim_mom6.mean()
    moms_model = np.array([mod_mom1, mod_mom2, mod_mom3, mod_mom4, 
                           mod_mom5, mod_mom6])
    err_vec = (moms_model - moms_data)/ moms_data
    return err_vec

# Create the ctiterion function
def crit(params, *args):
    alpha, rho, mu, sigma = params
    c_t, k_t, w_t, r_t, y_t, unif_vals, W = args
    err = err_vec(c_t, k_t, y_t, unif_vals, alpha, rho, mu, sigma)
    crit_val = err @ W @ err.T
    return crit_val

In [6]:
# Set the initial values and bounds of the parameters and the weighted matrix
alpha_init = 0.4
rho_init = 0.5
mu_init = 12
sigma_init = 0.5
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
bounds = ((0.01, 0.99), (-0.99, 0.99), (5, 14), (0.01, 1.1))
W = np.eye(6)

# Conduct SMM
smm_arg = (c_t, k_t, w_t, r_t, y_t, unif_vals, W)
result = opt.minimize(crit, params_init, args=(smm_arg), bounds = bounds)
alpha_smm, rho_smm, mu_smm, sigma_smm = result.x
print(result)
err = err_vec(c_t, k_t, y_t, unif_vals, alpha_smm, rho_smm, mu_smm, sigma_smm)
print(err)

      fun: 4.3073227920599241e-06
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  3.88496923e-04,   3.29135159e-05,   2.26052045e-05,
         7.07872906e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 320
      nit: 41
   status: 0
  success: True
        x: array([ 0.42103382,  0.92633018,  9.92846667,  0.08860187])
[  7.37109759e-04  -7.40062692e-04  -1.75193503e-03   8.65289678e-07
   2.72778046e-04  -2.69470315e-04]
